# ANTONYM-NEGATION-COMPARISON dataset

In [1]:
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser, pipeline
from helpers import prepare_dataset_nli, prepare_train_dataset_qa, \
    prepare_validation_dataset_qa, QuestionAnsweringTrainer, compute_accuracy
import os
import json
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

NUM_PREPROCESSING_WORKERS = 2

/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/__init__.py:284: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/stopwords-de.txt'>
  w = (w.strip() for w in w.read().split(","))
/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/__init__.py:284: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/stopwords-es.txt'>
  w = (w.strip() for w in w.read().split(","))
/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/__init__.py:284: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/stopwords-en.txt'>
  w = (w.strip() for w in w.read().split(","))
/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/__init__.py:284: Res

In [2]:
KEYS = ['id', 'title', 'context', 'question', 'answers']

### adapt checklist tooling to Huggingface SQuAD standard

In [3]:
editor = Editor()

/home/sambeck/code/checklist/checklist/editor.py:273: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/sambeck/code/checklist/checklist/data/lexicons/basic.json' mode='r' encoding='UTF-8'>
  self.lexicons.update(json.load(open(os.path.join(folder, f))))
/home/sambeck/code/checklist/checklist/editor.py:274: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/sambeck/code/checklist/checklist/data/names.json' mode='r' encoding='UTF-8'>
  self.data['names'] = json.load(open(os.path.join(cur_folder, 'data', 'names.json')))
/home/sambeck/code/checklist/checklist/editor.py:281: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sambeck/code/checklist/checklist/data/wikidata.pkl'>
  wikidata = pickle.load(open(os.path.join(cur_folder, 'data', 'wikidata.pkl'), 'rb'))


In [4]:
def create_data_dict(
    question_template, context_template, answer_template, lexicon_dict, title, n_samples=3000
):
    ret = editor.template({
        'question': question_template,
        'context': context_template, 
        },
        labels={'text': [answer_template]},
        **lexicon_dict,
        remove_duplicates=True,
        nsamples=n_samples,
    )
    print('Sample:')
    print(ret.data[0])
    print(ret.labels[0])

    ret.answers = ret.labels
    del ret.labels
    ret.question = [d['question'] for d in ret.data]
    ret.context = [d['context'] for d in ret.data]
    ret.id = []
    ret.title = []

    for i in range(len(ret.answers)):
        ret.answers[i]['answer_start'] = [ret.context[i].find(ret.answers[i]['text'][0])]
        ret.id.append(str(hex(hash(str(ret.data[i]))))[2:])
        ret.title.append(title)

    del ret.data
    return ret


In [5]:
antonyms = [
    ('progressive', 'conservative'),
    ('religious', 'secular'),
    ('positive', 'negative'),
    ('defensive', 'offensive'),
    ('rude',  'polite'),
    ('optimistic', 'pessimistic'),
    ('stupid', 'smart'),
    ('negative', 'positive'),
    ('unhappy', 'happy'),
    ('active', 'passive'),
    ('impatient', 'patient'),
    ('powerless', 'powerful'),
    ('visible', 'invisible'),
    ('fat', 'thin'),
    ('bad', 'good'),
    ('cautious', 'brave'),
    ('hopeful', 'hopeless'),
    ('insecure', 'secure'),
    ('humble', 'proud'),
    ('passive', 'active'),
    ('dependent', 'independent'),
    ('pessimistic', 'optimistic'),
    ('irresponsible', 'responsible'),
    ('courageous', 'fearful'),
    ('nice', 'mean'),
    ('young', 'old'),
    ('hot', 'cold'),
    ('loud', 'quiet'),
    ('unusual', 'normal'),
]


In [6]:
checklist_exs = {}

In [7]:
title='who_is_more_x'

x = create_data_dict(
    question_template='Who is {comparison} {madeupadj}?', 
    context_template='{male1} is {madeupadj}, but {male2} is more {madeupadj}.',
    answer_template='{male2}',
    lexicon_dict={
        'madeupadj': 'easy, educational, ordinary, academic, artistic, average, old, independent, entertaining, enjoyable, original, interesting, good, exciting, amateur, ideal, actual, experimental,  great, funny'.split(', '),
        'comparison': ['more', 'most', 'the most', ],
    }, 
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Who is more easy?', 'context': 'Nick is easy, but Jonathan is more easy.'}
{'text': ['Jonathan']}


In [8]:
title = 'who_is_more_x_er'

x = create_data_dict(
    question_template='Who is {comparison} {madeupadj}?', 
    context_template='{male1} is {madeupadj}, and {male2} is {madeupadj}er.',
    answer_template='{male2}',
    lexicon_dict={
        'madeupadj': 'old, innocent, great, young, sweet, dumb, proud'.split(', '),
        'comparison': ['more', 'most', 'the most', ],
    }, 
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Who is the most old?', 'context': 'Jack is old, and Andrew is older.'}
{'text': ['Andrew']}


In [9]:
title='who_is_less_x'

x = create_data_dict(
    question_template='Who is {comparison} {madeupadj}?', 
    context_template='{female1} is {madeupadj}{joiner} {female2} is {modifier}more {madeupadj}.',
    answer_template='{female1}',
    lexicon_dict={
        'madeupadj': 'easy, educational, ordinary, academic, artistic, average, old, independent, entertaining, enjoyable, original, interesting, good, exciting, amateur, ideal, experimental, innocent, interview, engaging, intelligent, interactive, bad, individual, great, funny'.split(', '),
        'joiner': [' and', '.', ' but', '; however,', ' although'],
        'modifier': ['way ', 'vastly ', '', 'much ', 'slightly ', 'entirely ', 'even '],
        'comparison': ['least', 'the least', 'less', 'not as',],
    }, 
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Who is not as independent?', 'context': 'Virginia is independent although Kathy is way more independent.'}
{'text': ['Virginia']}


In [10]:
title='who_is_less_x_er'

x = create_data_dict(
    question_template='Who is {comparison} {madeupadj}?', 
    context_template='{female1} is {madeupadj}{joiner} {female2} is {modifier}{madeupadj}er.',
    answer_template='{female1}',
    lexicon_dict={
        'madeupadj': 'old, great, young, sweet, dumb, proud, vast, green'.split(', '),
        'modifier': ['way ', 'vastly ', '', 'much ', 'slightly ', 'entirely ', 'even '], 
        'joiner': [' and', '.', ' but', '; however,', ' although'],
        'comparison': ['least', 'the least', 'less', 'not as',],
    }, 
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Who is the least young?', 'context': 'Carolyn is young although Kate is even younger.'}
{'text': ['Carolyn']}


In [11]:
title='who_is_less_antonym0'

x = create_data_dict(
    question_template='Which city is less {x[0]}?', 
    context_template='{city1} is {x[0]}. {city2} is {modifier}more {x[0]}.',
    answer_template='{city1}',
    lexicon_dict={'x': antonyms,
                  'modifier': ['way ', 'vastly ', '', 'much ', 'slightly ', 'entirely ', 'even ', 'a bit ']}, 
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which city is less irresponsible?', 'context': 'Philadelphia is irresponsible. St. Louis is slightly more irresponsible.'}
{'text': ['Philadelphia']}


In [12]:
title='who_is_less_antonym1'

x = create_data_dict(
    question_template='Which city is less {x[1]}?', 
    context_template='{city1} is {x[0]}. {city2} is {modifier}more {x[0]}.',
    answer_template='{city2}',
    lexicon_dict={'x': antonyms,
                  'modifier': ['way ', 'vastly ', '', 'much ', 'slightly ', 'entirely ', 'even ', 'a bit ']}, 
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which city is less offensive?', 'context': 'Oklahoma City is defensive. Corpus Christi is more defensive.'}
{'text': ['Corpus Christi']}


In [13]:
title='who_is_more_antonym0'

x = create_data_dict(
    question_template='Which city is {comparer} {x[0]}?', 
    context_template='{city1} is {x[0]}. {city2} is {modifier}more {x[0]}.',
    answer_template='{city2}',
    lexicon_dict={'x': antonyms,
                  'modifier': ['way ', 'vastly ', '', 'much ', 'slightly ', 'entirely ', 'even ', 'a bit ', 'quite a bit '],
                  'comparer': ['more', 'most', 'the most'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which city is more humble?', 'context': 'Virginia Beach is humble. North Las Vegas is much more humble.'}
{'text': ['North Las Vegas']}


In [14]:
title='who_is_more_antonym1'

x = create_data_dict(
    question_template='Which city is {comparer} {x[1]}?', 
    context_template='{city1} is {x[0]}. {city2} is {modifier}more {x[0]}.',
    answer_template='{city1}',
    lexicon_dict={'x': antonyms,
                  'modifier': ['way ', 'vastly ', '', 'much ', 'slightly ', 'entirely ', 'even ', 'a bit ', 'quite a bit '],
                  'comparer': ['more', 'most', 'the most'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which city is the most passive?', 'context': 'Glendale is active. Lexington is even more active.'}
{'text': ['Glendale']}


In [15]:
title='who_is_not_antonym1'

x = create_data_dict(
    question_template='Which city is {comparer} {x[1]}?', 
    context_template='{city1} is {x[0]}. {city2} is not {x[0]}.',
    answer_template='{city2}',
    lexicon_dict={'x': antonyms,
                  'comparer': ['more', 'most', 'the most', 'known for being'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which city is more cold?', 'context': 'Irvine is hot. Anchorage is not hot.'}
{'text': ['Anchorage']}


In [16]:
title='who_is_not_antonym0'

x = create_data_dict(
    question_template='Which city is {comparer} {x[0]}?', 
    context_template='{city1} is {x[1]}. {city2} is not {x[1]}.',
    answer_template='{city2}',
    lexicon_dict={'x': antonyms,
                  'comparer': ['more', 'most', 'the most', 'known for being'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which city is known for being courageous?', 'context': 'Milwaukee is fearful. Mesa is not fearful.'}
{'text': ['Mesa']}


In [17]:
title='antonym-least1'

x = create_data_dict(
    question_template='Which country is {comparer} {x[0]}?', 
    context_template='{country1} is {x[1]}. {country2} is {x[0]}.',
    answer_template='{country1}',
    lexicon_dict={'x': antonyms,
                  'modifier': ['way ', 'vastly ', '', 'much ', 'slightly ', 'entirely ', 'even ', 'a bit ', 'quite a bit '],
                  'comparer': ['less', 'least', 'the least', 'not known for being'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which country is the least progressive?', 'context': 'Belgium is conservative. Pakistan is progressive.'}
{'text': ['Belgium']}


In [18]:
title='antonym-least2'

x = create_data_dict(
    question_template='Which country is {comparer} {x[1]}?', 
    context_template='{country1} is {x[1]}. {country2} is {x[0]}.',
    answer_template='{country2}',
    lexicon_dict={'x': antonyms,
                  'modifier': ['way ', 'vastly ', '', 'much ', 'slightly ', 'entirely ', 'even ', 'a bit ', 'quite a bit '],
                  'comparer': ['less', 'least', 'the least', 'not known for being'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which country is least smart?', 'context': 'Equatorial Guinea is smart. Jamaica is stupid.'}
{'text': ['Jamaica']}


In [19]:
title='goldilocks'

x = create_data_dict(
    question_template='Which country is {comparer} {x[1]}?', 
    context_template='{country1} is {strongly1} {x[1]}. {country2} is {strongly2} {x[0]}. {country3} is in between the two.',
    answer_template='{country3}',
    lexicon_dict={'x': antonyms,
                  'strongly': ['strongly', 'super', 'extremely', 'severely', 'highly', 'ridiculously'],
                  'comparer': ['slightly', 'moderately'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which country is slightly smart?', 'context': 'Haiti is extremely smart. Sweden is strongly stupid. Switzerland is in between the two.'}
{'text': ['Switzerland']}


In [20]:
title='goldilocks-most-x1'

x = create_data_dict(
    question_template='Which country is {comparer} {x[1]}?', 
    context_template='{country1} is {strongly1} {x[1]}. {country2} is {strongly2} {x[0]}. {country3} is in between the two.',
    answer_template='{country1}',
    lexicon_dict={'x': antonyms,
                  'strongly': ['strongly ', 'super', 'extremely', 'severely', 'highly', 'ridiculously'],
                  'comparer': ['most', 'the most'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which country is the most passive?', 'context': 'China is extremely passive. Zimbabwe is severely active. Greece is in between the two.'}
{'text': ['China']}


In [21]:
title='goldilocks-most-x0'

x = create_data_dict(
    question_template='Which country is {comparer} {x[0]}?', 
    context_template='{country1} is {strongly1} {x[1]}. {country2} is {strongly2} {x[0]}. {country3} is in between the two.',
    answer_template='{country2}',
    lexicon_dict={'x': antonyms,
                  'strongly': ['strongly ', 'super', 'extremely', 'severely', 'highly', 'ridiculously'],
                  'comparer': ['most', 'the most'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which country is most hot?', 'context': 'Somalia is severely cold. Honduras is strongly  hot. Jamaica is in between the two.'}
{'text': ['Honduras']}


In [22]:
title='goldilocks-least-x1'

x = create_data_dict(
    question_template='Which country is {comparer} {x[1]}?', 
    context_template='{country1} is {strongly1} {x[1]}. {country2} is {strongly2} {x[0]}. {country3} is in between the two.',
    answer_template='{country2}',
    lexicon_dict={'x': antonyms,
                  'strongly': ['strongly', 'super', 'extremely', 'severely', 'highly', 'ridiculously'],
                  'comparer': ['least', 'the least'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which country is least old?', 'context': 'Federated States of Micronesia is super old. Equatorial Guinea is severely young. Iceland is in between the two.'}
{'text': ['Equatorial Guinea']}


In [23]:
title='goldilocks-least-x0'

x = create_data_dict(
    question_template='Which country is {comparer} {x[0]}?', 
    context_template='{country1} is {strongly1} {x[1]}. {country2} is {strongly2} {x[0]}. {country3} is in between the two.',
    answer_template='{country1}',
    lexicon_dict={'x': antonyms,
                  'strongly': ['strongly', 'super', 'extremely', 'severely', 'highly', 'ridiculously'],
                  'comparer': ['least', 'the least'],
                 },
    title=title,
)
checklist_exs[title] = x

Sample:
{'question': 'Which country is the least religious?', 'context': 'Finland is extremely secular. Syria is strongly religious. Bahrain is in between the two.'}
{'text': ['Finland']}


In [24]:
checklist_exs.keys()

dict_keys(['who_is_more_x', 'who_is_more_x_er', 'who_is_less_x', 'who_is_less_x_er', 'who_is_less_antonym0', 'who_is_less_antonym1', 'who_is_more_antonym0', 'who_is_more_antonym1', 'who_is_not_antonym1', 'who_is_not_antonym0', 'antonym-least1', 'antonym-least2', 'goldilocks', 'goldilocks-most-x1', 'goldilocks-most-x0', 'goldilocks-least-x1', 'goldilocks-least-x0'])

In [25]:
checklist_exs['antonym-least2'].keys()

dict_keys(['answers', 'question', 'context', 'id', 'title'])

In [26]:
from collections import defaultdict
lumped_data = defaultdict(list)
for k in KEYS:
    for example in checklist_exs.values():
        lumped_data[k].extend(example[k])

In [27]:
d = datasets.Dataset.from_dict(lumped_data)

In [28]:
d.save_to_disk('./experiments/antonym-negation/')